In [1]:
import geopandas as gpd
import cobra.postgres.interface as pgi
import cobra.utils.filehandling as fh
import cobra.utils.jobhandling as jh
import geopandas as gpd
import cobra.utils.geodatahandling as gdh

In [16]:
! ls /download/portugal

portugal-latest.osm.pbf


In [3]:
#download_file = 'https://download.geofabrik.de/europe/switzerland-latest.osm.pbf'
#country = 'Swiss'
#download_file = 'https://download.geofabrik.de/europe/great-britain-latest.osm.pbf'
#country = 'gb'
download_file = 'https://download.geofabrik.de/europe/portugal-latest.osm.pbf'
country = 'portugal'

In [2]:
fileman = fh.Filemanager()
jobman = jh.Jobmanager()

In [4]:
#jobman.delete_jobs()

In [23]:
fileman.download_plain_file(download_file, country)

In [4]:
datasets = fileman.get_data(datatype='OSM PBF')
dataset = datasets[datasets['Dataset'] == country].iloc[0]

In [5]:
dataset

Type                         OSM PBF
Dataset                     portugal
File         portugal-latest.osm.pbf
Path              /download/portugal
Size (MB)                     250.92
Name: 45, dtype: object

In [6]:
jobman.create_import_job_from_dataset(dataset, f'{country}power', style='power.style')

create_import_job_from_dataset
OSM PBF


In [7]:
jobman.get_jobs(df=True)

,UUID,Name,Job Type,Date created,Date started,Date finished,Priority,Status


In [13]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_line osm WHERE power IS NOT NULL"
pg_interface = pgi.PgInterface()
conn = pg_interface.get_connection()
powerlines = gpd.GeoDataFrame.from_postgis(sql, conn)

DatabaseError: Execution failed on sql 'SELECT osm.way AS geom, osm.* FROM portugalpower.planet_osm_line osm WHERE power IS NOT NULL': relation "portugalpower.planet_osm_line" does not exist
LINE 1: SELECT osm.way AS geom, osm.* FROM portugalpower.planet_osm_...
                                           ^


In [ ]:
powerlines.plot(figsize=(16,16))

In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_line osm WHERE power IS NOT NULL AND (power = 'line')"
jobman.create_new_pg_to_x(sql=sql, format='GPKG', filename=f'{country}_powerline.gpkg')

In [ ]:
jobman.get_jobs(df=True)

In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_point osm WHERE power IS NOT NULL"
pg_interface = pgi.PgInterface()
conn = pg_interface.get_connection()
powerpoints = gpd.GeoDataFrame.from_postgis(sql, conn)

In [ ]:
powerpoints.plot(figsize=(16, 16))

In [ ]:
sql = f"SELECT osm.way AS geom, osm.* FROM {country}power.planet_osm_point osm WHERE power IS NOT NULL"
jobman.create_new_pg_to_x(sql=sql, format='GPKG', filename=f'{country}_powerpoint.gpkg')

In [ ]:
jobman.get_jobs(df=True)